In [26]:
import pandas as pd
import networkx as nx
import simpy
from tqdm.auto import tqdm

class Lane:
    def __init__(self, 
                 id_: str, 
                 num_blocks: int, 
                 each_block_length: float,
                 each_block_capacity: int):
        self.id_ = id_
        self.length = num_blocks * each_block_length
        self.each_block_capacity = each_block_capacity
        self.capacity = num_blocks * each_block_capacity
        self.num_blocks = num_blocks
        self.stats = {i: 0 for i in range(num_blocks)}
        
    def __str__(self):
        return f"Lane {self.id_}: length={self.length}m, capacity={self.capacity}veh, blocks={self.num_blocks}"

    def __repr__(self):
        return self.__str__()

    def add_vehicle(self, block: int):
        self.stats[block] += 1
    
    def remove_vehicle(self, block: int):
        self.stats[block] -= 1
        
    def is_full(self, block: int) -> bool:
        return self.stats[block] >= self.each_block_capacity
            
            
class Intersection:
    def __init__(self, directions: list):
        self.num_directions = len(directions)
        self.directions = directions
        self.light_stats = {direction: "red" for direction in directions}
        self.stats = -1
        self.update_lights_according_to_logic()
        
    def __str__(self):
        return f"Intersection: {self.light_stats}"
    
    def __repr__(self):
        return self.__str__()
        
    def change_light(self, direction: str, color: str):
        self.light_stats[direction] = color

    def update_lights_according_to_logic(self):
        self.stats = (self.stats + 1) % self.num_directions
        for i, direction in enumerate(self.directions):
            if i == self.stats:
                self.change_light(direction, "green")
            else:
                self.change_light(direction, "red")
    
class Stats:
    def __init__(self):
        self.data = pd.DataFrame(columns=["player_id", "arrived_time", "departed_time", "wait_time"])
        self.detailed_data = pd.DataFrame(columns=["time", "player_id", "path_from", "path_to", "block"])
    
    def record_trip(self, player_id: str, arrived_time: int, departed_time: int, wait_time: int):
        new_record = {
            "player_id": player_id,
            "arrived_time": arrived_time,
            "departed_time": departed_time,
            "wait_time": wait_time,
        }
        self.data = pd.concat([self.data, pd.DataFrame([new_record])], ignore_index=True)
        
    def record_detailed(self, time: int, player_id: str, path_from: str, path_to: str, block: str):
        new_record = {
            "time": time,
            "player_id": player_id,
            "path_from": path_from,
            "path_to": path_to,
            "block": block
        }
        self.detailed_data = pd.concat([self.detailed_data, pd.DataFrame([new_record])], ignore_index=True)
        
class Player:
    def __init__(self, id_: str, source: str, destination: str, graph: nx.DiGraph, stats: Stats, arrival_time: int, env: simpy.Environment, logged: bool = False):
        self.id_ = id_
        self.source = source
        self.destination = destination
        self.wait_time = 0
        self.graph = graph
        self.stats = stats
        self.logged = logged
        self.arrival_time = arrival_time
        self.departed = 0
        self.done = False
        self.env = env
        self.shortest_path = self._shortest_path_seeker()
        
        self.current_path = [self.source, self._get_next_node(self.source)]
        self.intersection = self.graph.nodes[self.current_path[1]]["intersection"]
        self.lane = self.graph.edges[(self.current_path[0], self.current_path[1])]["lane"]
        self.lane.add_vehicle(block=self.lane.num_blocks - 1)
        self.block_position = self.lane.num_blocks - 1
        
    def _shortest_path_seeker(self):
        return nx.shortest_path(self.graph, source=self.source, target=self.destination, weight="travel_time")
        
    def _get_next_node(self, current_node: str) -> str:
        current_index = self.shortest_path.index(current_node)
        if current_index + 1 < len(self.shortest_path):
            return self.shortest_path[current_index + 1]
        return None

    def _move_forward(self):
        if self.lane.is_full(block=self.block_position):
            self.wait_time += 1
        else:
            self.lane.remove_vehicle(block=self.block_position)
            self.block_position -= 1
            self.lane.add_vehicle(block=self.block_position)
        self.log()
        
    def _move_intersection(self):
        if self.intersection.light_stats[self.current_path[0]] == "green":
            self.lane.remove_vehicle(block=self.block_position)
            self.current_path[0] = self.current_path[1]
            self.current_path[1] = self._get_next_node(self.current_path[0])
            if self.current_path[1] is not None:
                self.intersection = self.graph.nodes[self.current_path[1]]["intersection"]
                self.lane = self.graph.edges[(self.current_path[0], self.current_path[1])]["lane"]
                self.block_position = self.lane.num_blocks - 1
                self.lane.add_vehicle(block=self.block_position)
            else:
                self.departed = self.env.now
                self.done = True
                self.stats.record_trip(
                    player_id=self.id_,
                    arrived_time=self.arrival_time,
                    departed_time=self.departed,
                    wait_time=self.wait_time
                )
        else:
            self.wait_time += 1
        self.log()
        
    def make_decision(self):
        if self.done:
            return
        if self.block_position > 0:
            self._move_forward()
        elif self.block_position == 0:
            self._move_intersection()
        
    def log(self):
        if self.logged:
            self.stats.record_detailed(
                time=self.env.now,
                player_id=self.id_,
                path_from=self.current_path[0],
                path_to=self.current_path[1],
                block=self.block_position
            )

## LoadData Class Implementation

In [ ]:
class LoadData:
    def __init__(self):
        self.graph = None
        self.demand = None
    
    def load_network(self, network_file: str, position_file: str):
        data_xy = pd.read_csv(network_file)
        data_xy_pos = pd.read_csv(position_file, index_col=0)
        data_xy_pos = data_xy_pos.set_index('node').apply(lambda row: (row.x, row.y), axis=1).to_dict()
        speed = 80  # km/h
        data_xy_traveltime = {(int(row["from"]), int(row["to"])): row["length"]/(speed*1000/60) for _, row in data_xy.iterrows()} # minutes
        
        graph = nx.from_pandas_edgelist(data_xy, source="from", target="to", edge_attr=True, create_using=nx.DiGraph())
        nx.set_node_attributes(graph, data_xy_pos, 'pos')
        nx.set_edge_attributes(graph, data_xy_traveltime, 'travel_time')
        
        for edge in graph.edges(data=True):
            lane_id = 0
            num_blocks = edge[2]["length"] // 100
            lane = Lane(id_=lane_id, num_blocks=num_blocks, each_block_length=100, each_block_capacity=10)
            graph.edges[(edge[0], edge[1])]["lane"] = lane
            
        for node in graph.nodes:
            neighbors = list(graph.successors(node))
            intersection = Intersection(directions=neighbors)
            graph.nodes[node]["intersection"] = intersection
            
        self.graph = graph

    def load_demand(self, demand_file: str):
        self.demand = pd.read_csv(demand_file)

## Clock Class Implementation

In [ ]:
class Clock:
    def __init__(self, env: simpy.Environment, world: LoadData):
        self.env = env
        self.graph = world.graph
        self.demand = world.demand
        self.stats = Stats()
        self.player = {}

    def generate_player(self) -> simpy.events.Generator:
        with tqdm(total=self.duration, desc="Simulation Progress") as pbar:
            last_time = 0
            while True:
                current_time = self.env.now

                if current_time % 10 == 0:
                    for _, player in self.player.items():
                        player.make_decision()
                    for _, intersection in self.graph.nodes(data="intersection"):
                        intersection.update_lights_according_to_logic()

                arriving_players = self.demand[self.demand["arrival_time"] == current_time]
                for i, row in arriving_players.iterrows():
                    if i % 3 == 0:
                        logged = True
                    else:
                        logged = False
                    player = Player(
                        env=self.env,
                        id_=row["player_id"],
                        source=row["source"],
                        destination=row["destination"],
                        graph=self.graph,
                        stats=self.stats,
                        arrival_time=row["arrival_time"],
                        logged=logged
                    )
                    self.player[player.id_] = player

                # Update tqdm progress bar
                pbar.update(current_time - last_time)
                last_time = current_time

                yield self.env.timeout(1)
    
    def run_simulation(self, duration: int) -> simpy.events.Generator:
        self.duration = duration
        self.env.process(self.generate_player())
        self.env.run(until=duration)
        


In [36]:
network_file = "data/network/Network.csv"
position_file = "data/network/SiouxFalls_code_xy.csv"
demand_file = "data/network/demand.csv"

world = LoadData()
world.load_network(network_file=network_file, position_file=position_file)
world.load_demand(demand_file=demand_file)
clock = Clock(env=simpy.Environment(), world=world)
clock.run_simulation(duration=3600)

Simulation Progress:   0%|          | 0/3600 [00:00<?, ?it/s]

In [ ]:
clock.stats.detailed_data

,player_id,arrived_time,departed_time,wait_time
174,63,260,1750,9
187,1,2,1800,10
259,72,303,2090,9
278,98,370,2170,10
289,55,238,2220,9
299,141,482,2270,9
331,246,876,2390,12
335,206,710,2400,9
415,260,923,2710,9
446,296,1031,2830,10
